In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from helpers import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split

% matplotlib inline

In [2]:
# Import
df = pd.read_csv("~/real_estate/data/v6/yelp_supervised_df.csv")

In [3]:
current_time_features = [
    "90_day_treasury_bill_rate", "adult_count_lag", "bar_review_count", "coffee_review_count", 
    "mean_adjusted_income_lag", "mean_adjusted_income_with_dep_lag", "mean_fixed_mortgage_rate",
    "mean_hourly_earnings_adj", "mean_mortgage_interest_hh_lag", "mean_rating", "mean_real_estate_taxes_hh_lag",
    "median_sqft_value", "people_count_lag", "percent_decreasing", "percent_increasing", 
    "price_2_review_count", "price_3_review_count", "restaurant_review_count", "review_count", 
    "total_nonfarm_payroll", "zhvi_singlefam", "zri_sqft_value", "zhvi_condo"
]

target = "zhvi_condo_12_months"

In [4]:
for i in current_time_features:
    mo = i+"_mo"
    yr = i+"_yr"
    change_mo = (df[i] + 0.1)/(df[mo] + 0.1) - 1
    change_yr = (df[i] + 0.1)/(df[yr] + 0.1) - 1
    df[mo] = change_mo
    df[yr] = change_yr

df[target] = df[target]/df["zhvi_condo"] - 1

In [5]:
log_columns = ["median_sqft_value","zri_sqft_value","zhvi_condo","zhvi_singlefam","people_count_lag",
               "adult_count_lag","mean_adjusted_income_lag","mean_adjusted_income_with_dep_lag",
               "mean_real_estate_taxes_hh_lag","mean_mortgage_interest_hh_lag"]

for i in log_columns:
    df[i] = log_with_zeros(df[i].values)
    df = df.rename(columns={i:i+"_log"})

In [6]:
columns = ['postal_code','90_day_treasury_bill_rate','mean_adjusted_income_lag_log', 
 'mean_adjusted_income_with_dep_lag_log', 'mean_fixed_mortgage_rate', 
 'mean_hourly_earnings_adj', 'mean_mortgage_interest_hh_lag_log', 'mean_rating', 'mean_real_estate_taxes_hh_lag_log', 
 'median_sqft_value_log', 'month', 'people_count_lag_log', 'percent_decreasing', 'percent_increasing', 
 'price_2_review_count', 'price_3_review_count', 'restaurant_review_count', 'review_count', 'total_nonfarm_payroll', 
 'year', 'zhvi_condo_log', 'zhvi_singlefam_log', 
 'zri_sqft_value_log', 'adult_count_lag_mo', 'bar_review_count_mo', 'mean_mortgage_interest_hh_lag_mo', 
 'mean_fixed_mortgage_rate_mo', 'percent_decreasing_mo', 'zhvi_singlefam_mo', 'mean_adjusted_income_with_dep_lag_mo', 
 'restaurant_review_count_mo', 'zri_sqft_value_mo', 'percent_increasing_mo', 
 '90_day_treasury_bill_rate_mo', 'total_nonfarm_payroll_mo', 'mean_adjusted_income_lag_mo', 
 'mean_real_estate_taxes_hh_lag_mo', 'mean_hourly_earnings_adj_mo', 'review_count_mo', 'mean_rating_mo', 
 'people_count_lag_mo', 'price_3_review_count_mo', 'zhvi_condo_mo', 'coffee_review_count_mo', 'median_sqft_value_mo', 
 'price_2_review_count_mo', 'percent_decreasing_yr', 'coffee_review_count_yr', 
 'price_3_review_count_yr', 'price_2_review_count_yr', 'percent_increasing_yr', 'adult_count_lag_yr', 
 'mean_fixed_mortgage_rate_yr', 'mean_adjusted_income_lag_yr', 'zhvi_singlefam_yr', 'zri_sqft_value_yr', 
 'restaurant_review_count_yr', 'zhvi_condo_yr', 'mean_mortgage_interest_hh_lag_yr', 'total_nonfarm_payroll_yr', 
 'people_count_lag_yr', 'review_count_yr', 'mean_hourly_earnings_adj_yr', 'mean_rating_yr', 
 'bar_review_count_yr', 'median_sqft_value_yr', '90_day_treasury_bill_rate_yr', 'mean_real_estate_taxes_hh_lag_yr', 
 'mean_adjusted_income_with_dep_lag_yr', 'zhvi_condo_12_months']

df = df[columns]

In [7]:
train_df = df.loc[df["year"] < 2017]
test_df = df.loc[df["year"] == 2017]

features = df.columns[1:-1]
target = "zhvi_condo_12_months"

In [8]:
df.head()

,postal_code,90_day_treasury_bill_rate,mean_adjusted_income_lag_log,mean_adjusted_income_with_dep_lag_log,mean_fixed_mortgage_rate,mean_hourly_earnings_adj,mean_mortgage_interest_hh_lag_log,mean_rating,mean_real_estate_taxes_hh_lag_log,median_sqft_value_log,...,people_count_lag_yr,review_count_yr,mean_hourly_earnings_adj_yr,mean_rating_yr,bar_review_count_yr,median_sqft_value_yr,90_day_treasury_bill_rate_yr,mean_real_estate_taxes_hh_lag_yr,mean_adjusted_income_with_dep_lag_yr,zhvi_condo_12_months
0,2109,0.13,5.204049,5.045558,4.9525,22.87,1.580677,5.00,1.405052,6.318968,...,0.058382,0.909091,0.018174,0.00000,0.0,-0.048019,0.095238,0.008735,-0.260296,0.009221
1,2109,0.10,5.204049,5.045558,4.8360,22.87,1.580677,5.00,1.405052,6.320768,...,0.058382,0.909091,0.018174,0.00000,0.0,-0.049564,-0.200000,0.008735,-0.260296,0.011148
2,2109,0.06,5.204049,5.045558,4.8400,22.92,1.580677,5.00,1.405052,6.327937,...,0.058382,1.818182,0.018584,0.00000,0.0,-0.037794,-0.384615,0.008735,-0.260296,0.013040
3,2109,0.04,5.204049,5.045558,4.6375,22.99,1.580677,4.75,1.405052,6.335054,...,0.058382,0.952381,0.020327,-0.04902,0.0,-0.024217,-0.461538,0.008735,-0.260296,0.015713
4,2109,0.04,5.204049,5.045558,4.5100,23.01,1.580677,4.75,1.405052,6.335054,...,0.058382,0.952381,0.021211,-0.04902,0.0,-0.024217,-0.461538,0.008735,-0.260296,0.011006


In [9]:
# MinMax scale df
scaler = MinMaxScaler()
scaler.fit(train_df[features].values)

train_df_scaled = scaler.transform(train_df[features].values)
train_df_scaled = pd.DataFrame(train_df_scaled, columns=features)
train_df_scaled["target"] = train_df[target].values
train_df_scaled["postal_code"] = train_df["postal_code"].values

test_df_scaled = scaler.transform(test_df[features].values)
test_df_scaled = pd.DataFrame(test_df_scaled, columns=features)
test_df_scaled["target"] = test_df[target].values
test_df_scaled["postal_code"] = test_df["postal_code"].values

In [10]:
# Export
train_df_scaled.to_csv("~/real_estate/data/v6/yelp_train_df.csv",index=False)
test_df_scaled.to_csv("~/real_estate/data/v6/yelp_test_df.csv",index=False)
joblib.dump(scaler, "/home/gnazareths/real_estate/data/v6/yelp_scaler.pkl")

['/home/gnazareths/real_estate/data/v6/yelp_scaler.pkl']